In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv(r"C:\Users\ankit\Downloads\Zomato Dataset.csv")

Data Processing

In [3]:
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [4]:
df.shape

(45584, 20)

In [5]:
df.isnull().sum()


ID                                0
Delivery_person_ID                0
Delivery_person_Age            1854
Delivery_person_Ratings        1908
Restaurant_latitude               0
Restaurant_longitude              0
Delivery_location_latitude        0
Delivery_location_longitude       0
Order_Date                        0
Time_Orderd                    1731
Time_Order_picked                 0
Weather_conditions              616
Road_traffic_density            601
Vehicle_condition                 0
Type_of_order                     0
Type_of_vehicle                   0
multiple_deliveries             993
Festival                        228
City                           1200
Time_taken (min)                  0
dtype: int64

In [6]:
df.dtypes

ID                              object
Delivery_person_ID              object
Delivery_person_Age            float64
Delivery_person_Ratings        float64
Restaurant_latitude            float64
Restaurant_longitude           float64
Delivery_location_latitude     float64
Delivery_location_longitude    float64
Order_Date                      object
Time_Orderd                     object
Time_Order_picked               object
Weather_conditions              object
Road_traffic_density            object
Vehicle_condition                int64
Type_of_order                   object
Type_of_vehicle                 object
multiple_deliveries            float64
Festival                        object
City                            object
Time_taken (min)                 int64
dtype: object

Missing values handling

In [7]:
df['Delivery_person_Age'].fillna(df['Delivery_person_Age'].median(), inplace=True)
df['Delivery_person_Ratings'].fillna(df['Delivery_person_Ratings'].median(), inplace=True)
df['multiple_deliveries'].fillna(df['multiple_deliveries'].median(), inplace=True)


In [8]:
categorical_cols = ['Weather_conditions', 'Road_traffic_density', 'Festival', 'City', 'Time_Orderd']
for col in categorical_cols:
    df[col].fillna(df[col].mode()[0], inplace=True)


In [9]:
df.isnull().sum()


ID                             0
Delivery_person_ID             0
Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Order_Date                     0
Time_Orderd                    0
Time_Order_picked              0
Weather_conditions             0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           0
Time_taken (min)               0
dtype: int64

One Hot Encoding

In [10]:
df = pd.get_dummies(df, columns=[
    'Weather_conditions', 
    'Road_traffic_density', 
    'Type_of_order', 
    'Type_of_vehicle', 
    'Festival', 
    'City'
], drop_first=True)


In [11]:
df.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Vehicle_condition', 'multiple_deliveries',
       'Time_taken (min)', 'Weather_conditions_Fog',
       'Weather_conditions_Sandstorms', 'Weather_conditions_Stormy',
       'Weather_conditions_Sunny', 'Weather_conditions_Windy',
       'Road_traffic_density_Jam', 'Road_traffic_density_Low',
       'Road_traffic_density_Medium', 'Type_of_order_Drinks',
       'Type_of_order_Meal', 'Type_of_order_Snack',
       'Type_of_vehicle_electric_scooter', 'Type_of_vehicle_motorcycle',
       'Type_of_vehicle_scooter', 'Festival_Yes', 'City_Semi-Urban',
       'City_Urban'],
      dtype='object')

In [12]:
# Convert to datetime
df['Time_Orderd'] = pd.to_datetime(df['Time_Orderd'], format='%H:%M', errors='coerce')
df['Time_Order_picked'] = pd.to_datetime(df['Time_Order_picked'], format='%H:%M', errors='coerce')
df['Order_Date'] = pd.to_datetime(df['Order_Date'], errors='coerce')

# Fill missing times with mode
df['Time_Orderd'].fillna(df['Time_Orderd'].mode()[0], inplace=True)
df['Time_Order_picked'].fillna(df['Time_Order_picked'].mode()[0], inplace=True)

# Extract useful time features
df['order_hour'] = df['Time_Orderd'].dt.hour
df['pickup_hour'] = df['Time_Order_picked'].dt.hour
df['order_dayofweek'] = df['Order_Date'].dt.dayofweek  # 0=Monday

# Pickup delay in minutes
df['pickup_delay'] = (df['Time_Order_picked'] - df['Time_Orderd']).dt.total_seconds() / 60

# Drop original datetime columns
df.drop(['Time_Orderd', 'Time_Order_picked', 'Order_Date'], axis=1, inplace=True)


In [13]:
df.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Vehicle_condition',
       'multiple_deliveries', 'Time_taken (min)', 'Weather_conditions_Fog',
       'Weather_conditions_Sandstorms', 'Weather_conditions_Stormy',
       'Weather_conditions_Sunny', 'Weather_conditions_Windy',
       'Road_traffic_density_Jam', 'Road_traffic_density_Low',
       'Road_traffic_density_Medium', 'Type_of_order_Drinks',
       'Type_of_order_Meal', 'Type_of_order_Snack',
       'Type_of_vehicle_electric_scooter', 'Type_of_vehicle_motorcycle',
       'Type_of_vehicle_scooter', 'Festival_Yes', 'City_Semi-Urban',
       'City_Urban', 'order_hour', 'pickup_hour', 'order_dayofweek',
       'pickup_delay'],
      dtype='object')

In [14]:
df['travel_time_after_pickup'] = df['Time_taken (min)'] - df['pickup_delay']


In [15]:
df.dtypes

ID                                   object
Delivery_person_ID                   object
Delivery_person_Age                 float64
Delivery_person_Ratings             float64
Restaurant_latitude                 float64
Restaurant_longitude                float64
Delivery_location_latitude          float64
Delivery_location_longitude         float64
Vehicle_condition                     int64
multiple_deliveries                 float64
Time_taken (min)                      int64
Weather_conditions_Fog                 bool
Weather_conditions_Sandstorms          bool
Weather_conditions_Stormy              bool
Weather_conditions_Sunny               bool
Weather_conditions_Windy               bool
Road_traffic_density_Jam               bool
Road_traffic_density_Low               bool
Road_traffic_density_Medium            bool
Type_of_order_Drinks                   bool
Type_of_order_Meal                     bool
Type_of_order_Snack                    bool
Type_of_vehicle_electric_scooter

In [16]:
# Create weekend flag
df['is_weekend'] = df['order_dayofweek'].apply(lambda x: 1 if x in [5, 6] else 0)

# Drop the original day-of-week column
df.drop('order_dayofweek', axis=1, inplace=True)


Calculating distance using latitude and Longitude

In [17]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

df['distance_km'] = haversine(
    df['Restaurant_latitude'], df['Restaurant_longitude'],
    df['Delivery_location_latitude'], df['Delivery_location_longitude']
)


In [18]:
df.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Vehicle_condition',
       'multiple_deliveries', 'Time_taken (min)', 'Weather_conditions_Fog',
       'Weather_conditions_Sandstorms', 'Weather_conditions_Stormy',
       'Weather_conditions_Sunny', 'Weather_conditions_Windy',
       'Road_traffic_density_Jam', 'Road_traffic_density_Low',
       'Road_traffic_density_Medium', 'Type_of_order_Drinks',
       'Type_of_order_Meal', 'Type_of_order_Snack',
       'Type_of_vehicle_electric_scooter', 'Type_of_vehicle_motorcycle',
       'Type_of_vehicle_scooter', 'Festival_Yes', 'City_Semi-Urban',
       'City_Urban', 'order_hour', 'pickup_hour', 'pickup_delay',
       'travel_time_after_pickup', 'is_weekend', 'distance_km'],
      dtype='object')

Clipping the values to be between 0 and 180 minutes

In [19]:
# Keep only values between 0 and, say, 180 minutes (3 hours)
df = df[(df['travel_time_after_pickup'] >= 0) & (df['travel_time_after_pickup'] <= 180)] 


In [20]:
df['travel_time_after_pickup'].agg(['min', 'max', 'mean'])

min       0.000000
max     180.000000
mean     24.820622
Name: travel_time_after_pickup, dtype: float64

Predicting pickup delay

In [21]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# ==== Features at order placement ====
fixed = ['order_hour', 'is_weekend', 'Festival_Yes']
prefixes = ['Weather_conditions_', 'Road_traffic_density_', 'Type_of_order_', 'City_']
candidates = fixed + [c for c in df.columns if any(c.startswith(p) for p in prefixes)]
features_pd = [c for c in candidates if c in df.columns]

# ==== Target ====
target_pd = 'pickup_delay'
assert target_pd in df.columns, "pickup_delay column not found."
df[target_pd] = df[target_pd].clip(lower=0, upper=180)

X = df[features_pd].copy().fillna(0)
y = df[target_pd].astype(float).values

# Train/validation split (keep as your current setup)
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Numeric columns to scale for Ridge (leave one-hots as passthrough)
num_cols = [c for c in ['order_hour', 'is_weekend', 'Festival_Yes'] if c in X.columns]
ridge_pipe = Pipeline([
    ("scale_num", ColumnTransformer([("num", StandardScaler(), num_cols)], remainder="passthrough")),
    ("model", Ridge(alpha=2.0, random_state=42))
])

models = {
    "Ridge": ridge_pipe,
    "RandomForest": RandomForestRegressor(
        n_estimators=300, max_depth=14, min_samples_leaf=2, random_state=42, n_jobs=-1
    ),
    "GradientBoosting": GradientBoostingRegressor(
        n_estimators=400, learning_rate=0.05, max_depth=3, subsample=0.8, random_state=42
    ),
}

results = []
fitted = {}

for name, mdl in models.items():
    mdl.fit(X_train, y_train)
    y_hat = mdl.predict(X_val)
    mae = mean_absolute_error(y_val, y_hat)
    rmse = np.sqrt(mean_squared_error(y_val, y_hat))
    results.append({"Model": name, "MAE": mae, "RMSE": rmse})
    fitted[name] = mdl

# Leaderboard
leaderboard = pd.DataFrame(results).sort_values("MAE").reset_index(drop=True)
print("\n=== Pickup Delay: Model Comparison ===")
print(leaderboard)

# Top features per model
def show_top_features(name, mdl, k=10):
    print(f"\nTop features — {name}:")
    # If it's a pipeline (Ridge), pull the final estimator and transformed feature names
    if isinstance(mdl, Pipeline):
        final = mdl.named_steps["model"]
        pre = mdl.named_steps["scale_num"]
        # ColumnTransformer with remainder='passthrough' keeps original order:
        # first num_cols (scaled), then the rest unchanged.
        feat_names = list(X.columns)  # good enough for quick readout
        if hasattr(final, "coef_"):
            coefs = np.ravel(final.coef_)
            top = (pd.DataFrame({"Feature": feat_names, "Weight": coefs})
                   .assign(Abs=np.abs(coefs))
                   .sort_values("Abs", ascending=False)
                   .drop(columns=["Abs"])
                   .head(k))
            print(top)
            return
        mdl = final  # fallthrough to tree path if needed

    if hasattr(mdl, "feature_importances_"):
        top = (pd.DataFrame({"Feature": X.columns, "Importance": mdl.feature_importances_})
               .sort_values("Importance", ascending=False)
               .head(k))
        print(top)

for name, mdl in fitted.items():
    show_top_features(name, mdl, k=10)

# Use the best model to generate predictions for entire dataset
best_name = leaderboard.iloc[0]["Model"]
best_model = fitted[best_name]
df['pickup_delay_pred'] = best_model.predict(X)
print(f"\nBest model: {best_name}. Saved predictions to df['pickup_delay_pred'].")



=== Pickup Delay: Model Comparison ===
              Model       MAE      RMSE
0  GradientBoosting  3.857337  4.691018
1      RandomForest  3.929837  4.762237
2             Ridge  4.040974  4.873550

Top features — Ridge:
                          Feature    Weight
10    Road_traffic_density_Medium  0.888321
0                      order_hour -0.714673
9        Road_traffic_density_Low -0.598694
8        Road_traffic_density_Jam  0.560912
14                City_Semi-Urban  0.395944
6        Weather_conditions_Sunny -0.351064
15                     City_Urban -0.139909
3          Weather_conditions_Fog -0.134011
5       Weather_conditions_Stormy -0.126001
4   Weather_conditions_Sandstorms -0.123559

Top features — RandomForest:
                          Feature  Importance
0                      order_hour    0.449759
8        Road_traffic_density_Jam    0.203818
1                      is_weekend    0.047322
15                     City_Urban    0.040452
12             Type_of_order_Meal

Predicting after pickup time

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error

# ====== Feature set & target (same as yours) ======
features = [
    'Delivery_person_Age', 'Delivery_person_Ratings', 'Vehicle_condition',
    'multiple_deliveries',
    'Weather_conditions_Fog', 'Weather_conditions_Sandstorms',
    'Weather_conditions_Stormy', 'Weather_conditions_Sunny', 'Weather_conditions_Windy',
    'Road_traffic_density_Jam', 'Road_traffic_density_Low', 'Road_traffic_density_Medium',
    'Type_of_vehicle_electric_scooter', 'Type_of_vehicle_motorcycle', 'Type_of_vehicle_scooter',
    'Festival_Yes', 'City_Semi-Urban', 'City_Urban',
    'pickup_hour', 'is_weekend', 'distance_km'
]
target = 'travel_time_after_pickup'

X = df[features].copy()
y = df[target].astype(float).values

# Optional: simple fill for any residual NaNs
X = X.fillna(0)

# ====== Train/Test Split (same as yours) ======
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)

# ====== Models ======
models = {
    "Ridge": Ridge(alpha=2.0, random_state=42),
    "RandomForest": RandomForestRegressor(
        n_estimators=200, max_depth=12, random_state=42, n_jobs=-1
    ),
    "GradientBoosting": GradientBoostingRegressor(
        n_estimators=400, learning_rate=0.05, max_depth=3, subsample=0.8, random_state=42
    ),
}

results = []
importances = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    results.append({"Model": name, "MAE": mae, "RMSE": rmse})

    # Save feature importances for tree models
    if hasattr(model, "feature_importances_"):
        importances[name] = (
            pd.DataFrame({"Feature": features, "Importance": model.feature_importances_})
            .sort_values("Importance", ascending=False)
            .reset_index(drop=True)
        )

# ====== Leaderboard ======
leaderboard = pd.DataFrame(results).sort_values("MAE").reset_index(drop=True)
print("\n=== Post-pickup ETA: Model Comparison ===")
print(leaderboard)

# ====== Top features for tree models ======
for name, imp_df in importances.items():
    print(f"\nTop features — {name}:")
    print(imp_df.head(10))



=== Post-pickup ETA: Model Comparison ===
              Model        MAE       RMSE
0      RandomForest   9.886156  19.228755
1  GradientBoosting  10.314682  19.384220
2             Ridge  15.523516  28.842298

Top features — RandomForest:
                         Feature  Importance
0       Road_traffic_density_Low    0.421534
1                    pickup_hour    0.280789
2                    distance_km    0.081887
3        Delivery_person_Ratings    0.057776
4            Delivery_person_Age    0.056657
5              Vehicle_condition    0.021098
6       Weather_conditions_Sunny    0.013702
7            multiple_deliveries    0.012270
8         Weather_conditions_Fog    0.008869
9  Weather_conditions_Sandstorms    0.005316

Top features — GradientBoosting:
                    Feature  Importance
0  Road_traffic_density_Low    0.441065
1               pickup_hour    0.348911
2  Road_traffic_density_Jam    0.077502
3               distance_km    0.037207
4         Vehicle_condition   